### Imports

In [1]:
import numpy as np
import random
# import to do training
from tpg.trainer import Trainer
# import to run an agent (always needed)
from tpg.agent import Agent
# faster training
import multiprocessing as mp

### Grid World

In [6]:
# based (for now) on Downing's Figure 4.1
class GridWorldv1:
    def __init__(self, rows, cols, win_state, starting_state):  
        self.rows = rows
        self.cols = cols
        self.starting_state = starting_state
        self.win_state = win_state
        self.current_state = self.starting_state

    def sample_action (self):
        rand = random.uniform(0, 1)
        if (rand >= 0) and (rand < 0.25):
            return 0
        elif (rand >= 0.25) and (rand < 0.5):
            return 1
        elif (rand >= 0.5) and (rand < 0.75):
            return 2
        else:
            return 3
        
    def reset (self):
        self.current_state = self.starting_state
        return self.current_state
        
    def check_win (self):
        if self.current_state == self.win_state:
            return True
        return False

    # just reset for now...
    def close (self):
        self.current_state = self.starting_state
        return 1
    
    def step (self, action):
        # north
        if action == 0:
            next = (self.current_state[0] + 1, self.current_state[1])
        # south
        elif action == 1:
            next = (self.current_state[0] - 1, self.current_state[1])
        # east
        elif action == 2:
            next = (self.current_state[0], self.current_state[1] + 1)
        # west
        else:
            next = (self.current_state[0], self.current_state[1] - 1)

        terminate = False
        reward = 0
        # check if move is legal
        if (next[0] >= 0 and next[0] <= 9) and (next[1] >= 0 and next[1] <= 9):            
            illegal = 0
            for i in range(5):
                if next == (5, i):
                    illegal = 1
                    
            if (illegal == 0):
                self.current_state = next
                reward -= 0.01
                #print(self.current_state)
            else:
                #print('illegal move!')
                reward -= 1
                #terminate = True
        else:
            #print('out of bounds!')
            reward -= 1
            #terminate = True
        
        if self.check_win():
            reward += 100
            terminate = True
        
        return self.current_state, reward, terminate

### GridWorld Test

In [3]:
env = GridWorldv1(10, 10, (9, 0), (0, 0))
state = env.reset()
#action = agent.act(state)
action = env.sample_action()
state, reward, isDone = env.step(action)
print(state)
print(reward)
print(isDone)

(0, 0)
-1
True


### TPG Agent

In [8]:
def runAgent(args):
    agent = args[0]
    envName = args[1]
    scoreList = args[2]
    numEpisodes = args[3] # number of times to repeat game
    numFrames = args[4] 
    
    # skip if task already done by agent
    if agent.taskDone(envName):
        print('Agent #' + str(agent.agentNum) + ' can skip.')
        scoreList.append((agent.team.id, agent.team.outcomes))
        return
    
    env = GridWorldv1(10, 10, (9, 0), (0, 0))
    valActs = 4
    
    limit = 100
    
    # create environment object from above
    scoreTotal = 0
    for ep in range(numEpisodes):
        state = env.reset()
        scoreEp = 0
        if numEpisodes > 1:
            numRandFrames = random.randint(0,30)
            limitReached = 0
            for i in range(numFrames): # frame loop
                if i < numRandFrames:
                    env.step(env.sample_action())
                    continue
                    
                # action selection
                act = agent.act(state)
                
                state, reward, isDone = env.step(act)

                #print('State: ' + str(state))
                scoreEp += reward
                
                if isDone:
                    print('Win!')
                
                if i > limit:
                    print('Limit Reached!')
                    limitReached = True
                
                if isDone or limitReached:
                    break

            print('Agent #' + str(agent.agentNum) + 
              ' | Ep #' + str(ep) + ' | Score: ' + str(scoreEp))
    
        scoreTotal += scoreEp
    scoreTotal /= numEpisodes
    env.close()
    agent.reward(scoreTotal, envName)
    scoreList.append((agent.team.id, agent.team.outcomes))

### All Together Now!

In [9]:
acts = 4
# pAtomic equal to 1 so teams are single nodes
    # starting off, teams don't reference teams
trainer = Trainer(actions=acts, teamPopSize=10, pActAtom=0.999999999)

envName = 'GridWorldv1'
allScores = []
for gen in range(100):
    scoreList = []

    agents = trainer.getAgents()

    for agent in agents:
        runAgent([agent, envName, scoreList, 10, 1000])

    # apply scores, must do this when multiprocessing
    # because agents can't refer to trainer
    teams = trainer.applyScores(scoreList)

    trainer.evolve(tasks=[envName]) # go into next gen

    # an easier way to track stats than the above example
    scoreStats = trainer.fitnessStats

    allScores.append((scoreStats['min'], scoreStats['max'], scoreStats['average']))

    print('Gen: ' + str(gen))
    print('Results so far: ' + str(allScores))

print('Results:\nMin, Max, Avg')
for score in allScores:
    print(score[0],score[1],score[2])

Limit Reached!
Agent #0 | Ep #0 | Score: -100
Limit Reached!
Agent #0 | Ep #1 | Score: -0.7900000000000005
Limit Reached!
Agent #0 | Ep #2 | Score: -0.8300000000000005
Limit Reached!
Agent #0 | Ep #3 | Score: -2.000000000000001
Limit Reached!
Agent #0 | Ep #4 | Score: -0.7800000000000005
Limit Reached!
Agent #0 | Ep #5 | Score: -0.9000000000000006
Limit Reached!
Agent #0 | Ep #6 | Score: -0.8900000000000006
Limit Reached!
Agent #0 | Ep #7 | Score: -0.7200000000000004
Limit Reached!
Agent #0 | Ep #8 | Score: -0.8400000000000005
Limit Reached!
Agent #0 | Ep #9 | Score: -1.9400000000000008
Limit Reached!
Agent #1 | Ep #0 | Score: -77.03999999999999
Limit Reached!
Agent #1 | Ep #1 | Score: -73.08
Limit Reached!
Agent #1 | Ep #2 | Score: -89.09
Limit Reached!
Agent #1 | Ep #3 | Score: -68.07
Limit Reached!
Agent #1 | Ep #4 | Score: -70.07
Limit Reached!
Agent #1 | Ep #5 | Score: -94.02
Limit Reached!
Agent #1 | Ep #6 | Score: -96.02
Limit Reached!
Agent #1 | Ep #7 | Score: -90.0099999999999

Limit Reached!
Agent #7 | Ep #3 | Score: -71.03999999999999
Limit Reached!
Agent #7 | Ep #4 | Score: -78.03999999999999
Limit Reached!
Agent #7 | Ep #5 | Score: -102
Limit Reached!
Agent #7 | Ep #6 | Score: -80
Limit Reached!
Agent #7 | Ep #7 | Score: -79.02
Limit Reached!
Agent #7 | Ep #8 | Score: -74.03999999999999
Limit Reached!
Agent #7 | Ep #9 | Score: -78.00999999999999
Limit Reached!
Agent #8 | Ep #0 | Score: -78.05
Limit Reached!
Agent #8 | Ep #1 | Score: -97.02
Limit Reached!
Agent #8 | Ep #2 | Score: -97.03999999999999
Limit Reached!
Agent #8 | Ep #3 | Score: -75
Limit Reached!
Agent #8 | Ep #4 | Score: -86.02
Limit Reached!
Agent #8 | Ep #5 | Score: -72.08
Limit Reached!
Agent #8 | Ep #6 | Score: -79.03999999999999
Limit Reached!
Agent #8 | Ep #7 | Score: -71.03
Limit Reached!
Agent #8 | Ep #8 | Score: -82.02
Limit Reached!
Agent #8 | Ep #9 | Score: -76.08
Limit Reached!
Agent #9 | Ep #0 | Score: -69.09
Limit Reached!
Agent #9 | Ep #1 | Score: -86.08
Limit Reached!
Agent #9 

Limit Reached!
Agent #6 | Ep #3 | Score: -63.1
Limit Reached!
Agent #6 | Ep #4 | Score: -62.16
Limit Reached!
Agent #6 | Ep #5 | Score: -62.14
Limit Reached!
Agent #6 | Ep #6 | Score: -81.18
Limit Reached!
Agent #6 | Ep #7 | Score: -57.17
Limit Reached!
Agent #6 | Ep #8 | Score: -62.18
Limit Reached!
Agent #6 | Ep #9 | Score: -81.15
Limit Reached!
Agent #7 | Ep #0 | Score: -87.07
Limit Reached!
Agent #7 | Ep #1 | Score: -100
Limit Reached!
Agent #7 | Ep #2 | Score: -81.03
Limit Reached!
Agent #7 | Ep #3 | Score: -83.00999999999999
Limit Reached!
Agent #7 | Ep #4 | Score: -82.02
Limit Reached!
Agent #7 | Ep #5 | Score: -86.16000000000008
Limit Reached!
Agent #7 | Ep #6 | Score: -38.60000000000001
Limit Reached!
Agent #7 | Ep #7 | Score: -46.47
Limit Reached!
Agent #7 | Ep #8 | Score: -46.489999999999995
Limit Reached!
Agent #7 | Ep #9 | Score: -38.49000000000001
Limit Reached!
Agent #8 | Ep #0 | Score: -84.1
Limit Reached!
Agent #8 | Ep #1 | Score: -71.09
Limit Reached!
Agent #8 | Ep #2

Adding d89b5247-eff7-4f9c-affa-335db1eeced1 to trainer learners
Adding a3ace67d-55ce-41da-bcd9-fadc839374ad to trainer learners
Adding 9e3b7250-3c60-43ad-bde0-4cd1419b5438 to trainer learners
Adding ad2e1ac2-f045-4469-8a52-af62e5ff40e4 to trainer learners
Gen: 7
Results so far: [(-88.213, -10.969000000000001, -72.19709999999999), (-82.84, -10.969000000000001, -68.5793), (-90.314, -10.969000000000001, -68.46279999999999), (-87.23299999999998, -1.2550000000000203, -60.37099999999998), (-83.22899999999998, -1.2550000000000203, -55.20039999999999), (-82.434, -1.2550000000000203, -48.7364), (-82.43199999999999, -1.0080000000000007, -37.1239), (-84.287, -0.9710000000000006, -29.110899999999997)]
Agent #0 can skip.
Agent #1 can skip.
Agent #2 can skip.
Agent #3 can skip.
Agent #4 can skip.
Limit Reached!
Agent #5 | Ep #0 | Score: -89.00999999999999
Limit Reached!
Agent #5 | Ep #1 | Score: -84.00999999999999
Limit Reached!
Agent #5 | Ep #2 | Score: -84.00999999999999
Limit Reached!
Agent #5 | 

Limit Reached!
Agent #8 | Ep #0 | Score: -21.570000000000007
Limit Reached!
Agent #8 | Ep #1 | Score: -28.700000000000035
Limit Reached!
Agent #8 | Ep #2 | Score: -23.560000000000016
Limit Reached!
Agent #8 | Ep #3 | Score: -23.62000000000002
Limit Reached!
Agent #8 | Ep #4 | Score: -31.700000000000042
Limit Reached!
Agent #8 | Ep #5 | Score: -28.65000000000003
Limit Reached!
Agent #8 | Ep #6 | Score: -29.700000000000035
Limit Reached!
Agent #8 | Ep #7 | Score: -23.52000000000001
Limit Reached!
Agent #8 | Ep #8 | Score: -27.66000000000003
Limit Reached!
Agent #8 | Ep #9 | Score: -33.62000000000004
Limit Reached!
Agent #9 | Ep #0 | Score: -1.7400000000000007
Limit Reached!
Agent #9 | Ep #1 | Score: -1.7700000000000007
Limit Reached!
Agent #9 | Ep #2 | Score: -1.7900000000000007
Limit Reached!
Agent #9 | Ep #3 | Score: -0.9200000000000006
Limit Reached!
Agent #9 | Ep #4 | Score: -0.7200000000000004
Limit Reached!
Agent #9 | Ep #5 | Score: -1.8400000000000007
Limit Reached!
Agent #9 | Ep 

...

Limit Reached!
Agent #6 | Ep #2 | Score: -69.03999999999999
Limit Reached!
Agent #6 | Ep #3 | Score: -98.00999999999999
Limit Reached!
Agent #6 | Ep #4 | Score: -81.00999999999999
Limit Reached!
Agent #6 | Ep #5 | Score: -84
Limit Reached!
Agent #6 | Ep #6 | Score: -88.03
Limit Reached!
Agent #6 | Ep #7 | Score: -85.03
Limit Reached!
Agent #6 | Ep #8 | Score: -88
Limit Reached!
Agent #6 | Ep #9 | Score: -95
Limit Reached!
Agent #7 | Ep #0 | Score: -78.15
Limit Reached!
Agent #7 | Ep #1 | Score: -59.16
Limit Reached!
Agent #7 | Ep #2 | Score: -75.15
Limit Reached!
Agent #7 | Ep #3 | Score: -74.16
Limit Reached!
Agent #7 | Ep #4 | Score: -57.15
Limit Reached!
Agent #7 | Ep #5 | Score: -67.12
Limit Reached!
Agent #7 | Ep #6 | Score: -82.18
Limit Reached!
Agent #7 | Ep #7 | Score: -71.14
Limit Reached!
Agent #7 | Ep #8 | Score: -81.15
Limit Reached!
Agent #7 | Ep #9 | Score: -73.16
Limit Reached!
Agent #8 | Ep #0 | Score: -1.9800000000000009
Limit Reached!
Agent #8 | Ep #1 | Score: -0.8100

Limit Reached!
Agent #9 | Ep #0 | Score: -0.8800000000000006
Limit Reached!
Agent #9 | Ep #1 | Score: -0.8100000000000005
Limit Reached!
Agent #9 | Ep #2 | Score: -0.9800000000000006
Limit Reached!
Agent #9 | Ep #3 | Score: -0.9900000000000007
Limit Reached!
Agent #9 | Ep #4 | Score: -0.8400000000000005
Limit Reached!
Agent #9 | Ep #5 | Score: -2.0100000000000007
Limit Reached!
Agent #9 | Ep #6 | Score: -0.8800000000000006
Limit Reached!
Agent #9 | Ep #7 | Score: -1.7300000000000006
Limit Reached!
Agent #9 | Ep #8 | Score: -0.8700000000000006
Limit Reached!
Agent #9 | Ep #9 | Score: -0.8800000000000006
Adding 49bd4ff8-f6a2-4b31-9864-ae6a5ae180a6 to trainer learners
Adding d113384b-b244-47d2-a579-2d71162362d9 to trainer learners
Adding cf2b1821-f514-4418-b85b-a1319116aa59 to trainer learners
Adding 067d0b03-05ee-4b11-9171-3b6f248c79d9 to trainer learners
Adding 6a007747-33f0-402e-af22-c70856a5ca1f to trainer learners
Adding c7e92d15-ad89-423d-bacd-37012a70b712 to trainer learners
Gen: 9

Limit Reached!
Agent #6 | Ep #6 | Score: -89.00999999999999
Limit Reached!
Agent #6 | Ep #7 | Score: -90.03999999999999
Limit Reached!
Agent #6 | Ep #8 | Score: -93.02
Limit Reached!
Agent #6 | Ep #9 | Score: -88.03999999999999
Limit Reached!
Agent #7 | Ep #0 | Score: -85.07
Limit Reached!
Agent #7 | Ep #1 | Score: -77.05
Limit Reached!
Agent #7 | Ep #2 | Score: -89.06
Limit Reached!
Agent #7 | Ep #3 | Score: -75.08
Limit Reached!
Agent #7 | Ep #4 | Score: -90.03999999999999
Limit Reached!
Agent #7 | Ep #5 | Score: -98.03
Limit Reached!
Agent #7 | Ep #6 | Score: -83.05
Limit Reached!
Agent #7 | Ep #7 | Score: -72.05
Limit Reached!
Agent #7 | Ep #8 | Score: -78.05
Limit Reached!
Agent #7 | Ep #9 | Score: -75.06
Limit Reached!
Agent #8 | Ep #0 | Score: -88
Limit Reached!
Agent #8 | Ep #1 | Score: -100
Limit Reached!
Agent #8 | Ep #2 | Score: -67.13
Limit Reached!
Agent #8 | Ep #3 | Score: -71.06
Limit Reached!
Agent #8 | Ep #4 | Score: -77.09
Limit Reached!
Agent #8 | Ep #5 | Score: -86.

Limit Reached!
Agent #9 | Ep #6 | Score: -90.03999999999999
Limit Reached!
Agent #9 | Ep #7 | Score: -85.03999999999999
Limit Reached!
Agent #9 | Ep #8 | Score: -80.18
Limit Reached!
Agent #9 | Ep #9 | Score: -80
Adding 8efc9e66-dc94-4053-ae1e-210b33e82d6f to trainer learners
Adding 5fbae267-e9af-452b-9c28-d6c88b516137 to trainer learners
Adding 0677f4f7-f177-4a0f-876a-5f0d3c95299e to trainer learners
Adding 944c5097-6f27-45a6-80c2-f65df3d7a09c to trainer learners
Adding ddb76e00-a334-41f9-9cab-653ebdf8b863 to trainer learners
Adding 7413c6b0-16b4-4494-a50c-054596a46e1c to trainer learners
Adding c44759b6-5853-44c5-a8e4-adc352956b14 to trainer learners
Adding 67201495-6fc4-453a-887d-45e4bfe6f9a9 to trainer learners
Gen: 97
Results so far: [(-88.213, -10.969000000000001, -72.19709999999999), (-82.84, -10.969000000000001, -68.5793), (-90.314, -10.969000000000001, -68.46279999999999), (-87.23299999999998, -1.2550000000000203, -60.37099999999998), (-83.22899999999998, -1.2550000000000203, 

Limit Reached!
Agent #7 | Ep #8 | Score: -0.9900000000000007
Limit Reached!
Agent #7 | Ep #9 | Score: -1.8900000000000008
Limit Reached!
Agent #8 | Ep #0 | Score: -59.15
Limit Reached!
Agent #8 | Ep #1 | Score: -63.14
Limit Reached!
Agent #8 | Ep #2 | Score: -65.12
Limit Reached!
Agent #8 | Ep #3 | Score: -64.16
Limit Reached!
Agent #8 | Ep #4 | Score: -87.15
Limit Reached!
Agent #8 | Ep #5 | Score: -86.1
Limit Reached!
Agent #8 | Ep #6 | Score: -71.1
Limit Reached!
Agent #8 | Ep #7 | Score: -68.07
Limit Reached!
Agent #8 | Ep #8 | Score: -68.08
Limit Reached!
Agent #8 | Ep #9 | Score: -74.16
Limit Reached!
Agent #9 | Ep #0 | Score: -0.9000000000000006
Limit Reached!
Agent #9 | Ep #1 | Score: -0.7500000000000004
Limit Reached!
Agent #9 | Ep #2 | Score: -0.9900000000000007
Limit Reached!
Agent #9 | Ep #3 | Score: -2.0100000000000007
Limit Reached!
Agent #9 | Ep #4 | Score: -0.7500000000000004
Limit Reached!
Agent #9 | Ep #5 | Score: -0.7900000000000005
Limit Reached!
Agent #9 | Ep #6 | 